In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from vae_methods import Sampling
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split

In [42]:
data_path = "../SWaT_Data"
with open(f"{data_path}/scalerX_0.pkl", "rb") as f:
    scaler = pickle.load(f)

df = pd.read_csv(f"{data_path}/SWaT_Dataset_Attack_v0.csv")

# Relabel
df.replace(["A ttack", "Attack", "Normal"], [1, 1, 0], inplace=True)

# df

In [3]:
df.columns

Index([' Timestamp', 'FIT101', 'LIT101', ' MV101', 'P101', 'P102', ' AIT201',
       'AIT202', 'AIT203', 'FIT201', ' MV201', ' P201', ' P202', 'P203',
       ' P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301',
       'MV302', ' MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402',
       'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501',
       'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504',
       'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602',
       'P603', 'Normal/Attack', 'Unnamed: 53', 'Unnamed: 54'],
      dtype='object')

In [43]:
df_data = df.drop(columns=[df.columns[0], df.columns[-1], df.columns[-2], df.columns[-3]]).dropna()

In [44]:
df_attack = scaler.transform(df_data)
labels = df['Normal/Attack']

x_train, x_test, y_train, y_test = train_test_split(df_attack, labels, test_size=0.4, random_state=0)

In [45]:
x_train.shape

(269951, 51)

In [46]:
y_train.shape

(269951,)

In [47]:
x_test.shape

(179968, 51)

In [49]:
y_test.shape

(179968,)

In [36]:
del df_data
del df
del labels
del df_attack

In [9]:
input_dim = x_train.shape[-1]

In [10]:
encoder = tf.keras.models.load_model('VAE_Encoder_100422.h5', custom_objects={"Sampling": Sampling})

In [11]:
encoder.trainable = False

In [21]:
encoder.save_weights("VAE_Encoder_100422_weights.ckpt")

In [24]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=11,
    restore_best_weights=True,
)

reduce_lr_calllback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.15,
    patience=5,
    verbose=0,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

In [ ]:
def create_classifier(encoder, input_dim):
    
    inputs = tf.keras.Input(shape=(input_dim,))
    z_mean, z_logvar, z = encoder(inputs, training=False)
    latent = tf.keras.layers.Concatenate()([z_mean, z_logvar])
    dense1 = tf.keras.layers.Dense(30, activation='tanh')(latent)
    dense2 = tf.keras.layers.Dense(15, activation='relu')(dense1)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)
    
    model = tf.keras.Model(inputs, outputs, name='Classifier')
    
    return model

In [56]:
latent_dim = 45

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    
    inputs_encoder_vae = tf.keras.Input(shape=(input_dim,))

    dense1_vae = tf.keras.layers.Dense(input_dim - 1, activation='tanh')(inputs_encoder_vae)
    dense2_vae = tf.keras.layers.Dense(input_dim - 5, activation='relu')(dense1_vae)
#         dense2_vae = tf.keras.layers.Dense(input_dim - 5, activation='relu', kernel_regularizer='l2')(dense1_vae)

    z_mean = tf.keras.layers.Dense(latent_dim, name="z_mean")(dense2_vae)
    z_log_var = tf.keras.layers.Dense(latent_dim, name="z_log_var")(dense2_vae)
    z = Sampling()([z_mean, z_log_var])

    encoder = tf.keras.Model(inputs=inputs_encoder_vae,
                            outputs=[z_mean, z_log_var, z],
                            name='VAE_encoder')
    
    encoder.load_weights("VAE_Encoder_100422_weights.ckpt")
    
    encoder.trainable = False
    
    inputs = tf.keras.Input(shape=(input_dim,))
    z_mean, z_logvar, z = encoder(inputs, training=False)
    latent = tf.keras.layers.Concatenate()([z_mean, z_logvar])
    dense1 = tf.keras.layers.Dense(30, activation='tanh')(latent)
    dense2 = tf.keras.layers.Dense(15, activation='relu')(dense1)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)
#     outputs = tf.keras.layers.Dense(1, activation='relu')(dense2)
    
    model = tf.keras.Model(inputs, outputs, name='Classifier')
    
    model.compile(loss=tfa.losses.SigmoidFocalCrossEntropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1),
                 metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
#                   tf.keras.losses.BinaryCrossentropy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [57]:
hist = model.fit(x_train, y_train,
                 epochs=100,
                 validation_split=0.15,
                 batch_size=128,
                 callbacks=[early_stopping_callback, reduce_lr_calllback])

Epoch 1/100
1793/1793 [==============================] - 18s 7ms/step - loss: 0.0178 - accuracy: 0.9471 - precision_6: 0.9329 - recall_6: 0.6092 - val_loss: 0.0139 - val_accuracy: 0.9577 - val_precision_6: 0.9917 - val_recall_6: 0.6565
Epoch 2/100
1793/1793 [==============================] - 11s 6ms/step - loss: 0.0134 - accuracy: 0.9587 - precision_6: 0.9879 - recall_6: 0.6690 - val_loss: 0.0122 - val_accuracy: 0.9611 - val_precision_6: 0.9877 - val_recall_6: 0.6869
Epoch 3/100
1793/1793 [==============================] - 11s 6ms/step - loss: 0.0123 - accuracy: 0.9629 - precision_6: 0.9851 - recall_6: 0.7061 - val_loss: 0.0125 - val_accuracy: 0.9663 - val_precision_6: 0.9833 - val_recall_6: 0.7338
Epoch 4/100
1793/1793 [==============================] - 11s 6ms/step - loss: 0.0117 - accuracy: 0.9646 - precision_6: 0.9851 - recall_6: 0.7206 - val_loss: 0.0112 - val_accuracy: 0.9673 - val_precision_6: 0.9799 - val_recall_6: 0.7449
Epoch 5/100
1793/1793 [==============================] -

In [58]:
model.evaluate(x_test, y_test)

5624/5624 [==============================] - 25s 4ms/step - loss: 0.0058 - accuracy: 0.9794 - precision_6: 0.9885 - recall_6: 0.8392


[0.005796907935291529,
 0.9793630242347717,
 0.9884752631187439,
 0.8391823768615723]

In [59]:
encoder.trainable = True

In [60]:
hist2 = model.fit(x_train, y_train,
                 epochs=100,
                 validation_split=0.15,
                 batch_size=128,
                 callbacks=[early_stopping_callback, reduce_lr_calllback])

Epoch 1/100
1793/1793 [==============================] - 11s 6ms/step - loss: 0.0057 - accuracy: 0.9796 - precision_6: 0.9876 - recall_6: 0.8435 - val_loss: 0.0055 - val_accuracy: 0.9805 - val_precision_6: 0.9879 - val_recall_6: 0.8493
Epoch 2/100
1793/1793 [==============================] - 11s 6ms/step - loss: 0.0057 - accuracy: 0.9796 - precision_6: 0.9877 - recall_6: 0.8429 - val_loss: 0.0055 - val_accuracy: 0.9805 - val_precision_6: 0.9879 - val_recall_6: 0.8497
Epoch 3/100
1793/1793 [==============================] - 11s 6ms/step - loss: 0.0057 - accuracy: 0.9796 - precision_6: 0.9877 - recall_6: 0.8430 - val_loss: 0.0055 - val_accuracy: 0.9805 - val_precision_6: 0.9879 - val_recall_6: 0.8497
Epoch 4/100
1793/1793 [==============================] - 11s 6ms/step - loss: 0.0057 - accuracy: 0.9796 - precision_6: 0.9877 - recall_6: 0.8430 - val_loss: 0.0055 - val_accuracy: 0.9805 - val_precision_6: 0.9879 - val_recall_6: 0.8491
Epoch 5/100
1793/1793 [==============================] -

In [61]:
model.evaluate(x_test, y_test)

5624/5624 [==============================] - 24s 4ms/step - loss: 0.0058 - accuracy: 0.9792 - precision_6: 0.9885 - recall_6: 0.8381


[0.005796505603939295,
 0.9792407751083374,
 0.9885144829750061,
 0.8381258845329285]

In [62]:
model.save("Classifier2_100422.h5")